#### CE-240 - Database Systems Design
##### Analysis of student fluency - AFA-BD
 - Feature: Spark SQL instructions
 - Author: Strauss
 - Date: 2024-05-31

#### Main Goal

In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import rand,when
from pyspark.sql.functions import lit
from pyspark.sql import functions as f
from pyspark.sql.functions import row_number, monotonically_increasing_id 
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, IntegerType, StringType
import random
import warnings
import sqlite3
warnings.filterwarnings('ignore')

In [2]:
#sc.stop()
sc = pyspark.SparkContext('local[*]')
sqlc = SQLContext(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/03 02:13:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pth = 'file:////home/jovyan/work/data'

fle_std = '{0}/tb-students.csv'.format(pth)
fle_pro = '{0}/tb-professors.csv'.format(pth)
fle_sch = '{0}/tb-inep.csv'.format(pth)

#### 1 - Loading Data

##### 1.1 - Loading the Student dataframe

In [4]:
rdd = sc.textFile(fle_std)
dfs_std = sqlc.read.csv(rdd, header=True, inferSchema=True, sep=',')
print('Dataframe Spark possui {0} linhas'.format(dfs_std.count()))

24/06/03 02:13:29 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker
24/06/03 02:13:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , alu_id, alu_nome, alu_sexo, alu_endereco, alu_cidade, alu_cep, alu_uf, alu_telefone, alu_email, alu_data_nascimento, pro_latitude, pro_longitude, tur_id, progresso_alfabetizacao
 Schema: _c0, alu_id, alu_nome, alu_sexo, alu_endereco, alu_cidade, alu_cep, alu_uf, alu_telefone, alu_email, alu_data_nascimento, pro_latitude, pro_longitude, tur_id, progresso_alfabetizacao
Expected: _c0 but found: 
CSV source: [value: string]


Dataframe Spark possui 1000000 linhas


In [5]:
dfs_std.show(n=1, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------------------
 _c0                     | 0                                                                 
 alu_id                  | 3011133716                                                        
 alu_nome                | Enrico da Rocha                                                   
 alu_sexo                | F                                                                 
 alu_endereco            | Travessa de Nunes, 34 Mangueiras 77558573 Barbosa de Pereira / PI 
 alu_cidade              | Pinto                                                             
 alu_cep                 | 77137055                                                          
 alu_uf                  | RR                                                                
 alu_telefone            | (061) 0341-7969                                                   
 alu_email               | evelyncaldeira@example.org       

24/06/03 02:13:36 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 5 (TID 14): Attempting to kill Python Worker


##### 1.2 - Loading the Professors dataframe

In [6]:
rdd = sc.textFile(fle_pro)
dfs_pro = sqlc.read.csv(rdd, header=True, inferSchema=True, sep=',')
print('Dataframe Spark possui {0} linhas'.format(dfs_pro.count()))

24/06/03 02:13:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , pro_id, pro_nome, ultimo_nome, pro_senha, pro_sexo, pro_escolaridade, pro_credenciado, pro_endereco, pro_cidade, pro_cep, pro_uf, pro_pais, pro_telefone, pro_email, pro_nascimento, pro_latitude, pro_longitude, pro_salario
 Schema: _c0, pro_id, pro_nome, ultimo_nome, pro_senha, pro_sexo, pro_escolaridade, pro_credenciado, pro_endereco, pro_cidade, pro_cep, pro_uf, pro_pais, pro_telefone, pro_email, pro_nascimento, pro_latitude, pro_longitude, pro_salario
Expected: _c0 but found: 
CSV source: [value: string]


Dataframe Spark possui 14000 linhas


In [7]:
dfs_pro.show(n=1, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------------
 _c0              | 0                                                              
 pro_id           | 46631645                                                       
 pro_nome         | Sr. Vicente Azevedo                                            
 ultimo_nome      | Freitas                                                        
 pro_senha        | N#M4NiMHR^                                                     
 pro_sexo         | F                                                              
 pro_escolaridade | 3                                                              
 pro_credenciado  | 0                                                              
 pro_endereco     | Quadra Nogueira, 35 Boa União 1ª Seção 09964-112 da Rocha / AM 
 pro_cidade       | Silva de Sales                                                 
 pro_cep          | 09074728                                                

##### 1.3 - Loading the Schools dataframe

In [50]:
rdd = sc.textFile(fle_sch)
dfs_sch = sqlc.read.csv(rdd, header=True, inferSchema=True, sep=';')
print('Dataframe Spark possui {0} linhas'.format(dfs_sch.count()))

24/06/03 02:26:06 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 74 (TID 123): Attempting to kill Python Worker
24/06/03 02:26:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: cod_inep, escola, uf, municipio, endereco, latitude, longitude, telefone, localizacao, localidade, categoria, dependencia, categoria, conveniada, regulamentacao, porte, modalidade, ofertas, restricao, situacao
 Schema: cod_inep, escola, uf, municipio, endereco, latitude, longitude, telefone, localizacao, localidade, categoria10, dependencia, categoria12, conveniada, regulamentacao, porte, modalidade, ofertas, restricao, situacao
Expected: categoria10 but found: categoria
CSV source: [value: string]


Dataframe Spark possui 180230 linhas


In [51]:
dfs_sch.show(n=1, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------------------
 cod_inep       | 11000023                                                                   
 escola         | EEEE ABNAEL MACHADO DE LIMA - CENE                                         
 uf             | RO                                                                         
 municipio      | Porto Velho                                                                
 endereco       | AVENIDA AMAZONAS, 6492 ZONA LESTE. TIRADENTES. 76824-556 Porto Velho - RO. 
 latitude       | -8.758.459                                                                 
 longitude      | -638.540.109                                                               
 telefone       | (69) 992222374                                                             
 localizacao    | Urbana                                                                     
 localidade     | A escola não está em área de localização d

24/06/03 02:26:13 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 79 (TID 131): Attempting to kill Python Worker


#### 2 - SQL Instructions

##### 2.1 - Create index relationship

In [52]:
n_pro = 14000
n_std = 1000000
cod_schools  = dfs_sch.rdd.map(lambda x: x['cod_inep']).collect()

In [53]:
labels_udf = f.udf(lambda indx: cod_schools[indx-1], StringType())

##### - Professors

In [54]:
cod_schools = cod_schools[:n_pro]
dfs_pro = dfs_pro.withColumn("id", row_number().over(Window.orderBy(monotonically_increasing_id())))
dfs_pro = dfs_pro.withColumn('esc_cod', labels_udf('id')) 

##### - Students

In [55]:
cod_schools = cod_schools[:n_std]
dfs_std = dfs_std.withColumn("id", row_number().over(Window.orderBy(monotonically_increasing_id())))
dfs_std = dfs_std.withColumn('esc_cod', labels_udf('id')) 
dfs_std = dfs_std.withColumn('idade', (f.months_between(f.current_date(), f.col('alu_data_nascimento')) / 12).cast('int'))

##### 2.2 - Simple Queries 

In [56]:
dfs_std.createOrReplaceTempView("tb_student")
dfs_pro.createOrReplaceTempView("tb_professor")
dfs_sch.createOrReplaceTempView("tb_school")

##### 2.2.1 - Students

In [47]:
sqlc.sql("""SELECT esc_cod, alu_nome, alu_cidade, alu_uf, alu_data_nascimento, progresso_alfabetizacao, idade
         FROM tb_student""").show(12)

24/06/03 02:24:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/03 02:24:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/03 02:24:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+--------------------+------------------+------+-------------------+-----------------------+-----+
| esc_cod|            alu_nome|        alu_cidade|alu_uf|alu_data_nascimento|progresso_alfabetizacao|idade|
+--------+--------------------+------------------+------+-------------------+-----------------------+-----+
|11000023|     Enrico da Rocha|             Pinto|    RR|         2017-09-15|                      1|    6|
|11000040|Maria Sophia Costela|  Novaes de Castro|    TO|         2016-07-23|                      8|    7|
|11000058|     Letícia Cardoso|   Vieira do Campo|    AM|         2015-07-11|                      9|    8|
|11000082|        Sophia Nunes|    Alves do Galho|    GO|         2017-12-05|                      8|    6|
|11000104|      Melissa Santos|           Martins|    PB|         2017-08-22|                      7|    6|
|11000198|      Lavínia Moraes|           Freitas|    MT|         2015-04-04|                      2|    9|
|11000201|Srta. Ana Lívia N.

##### 2.2.2 - Proferssors

In [16]:
sqlc.sql("""SELECT esc_cod, pro_nome, pro_cidade, pro_uf, pro_email, pro_salario, pro_credenciado, pro_senha
         FROM tb_professor""").show(12)

24/06/03 02:13:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/03 02:13:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/03 02:13:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+--------------------+--------------------+------+--------------------+-----------+---------------+----------+
| esc_cod|            pro_nome|          pro_cidade|pro_uf|           pro_email|pro_salario|pro_credenciado| pro_senha|
+--------+--------------------+--------------------+------+--------------------+-----------+---------------+----------+
|11000023| Sr. Vicente Azevedo|      Silva de Sales|    PE|jesusvitoria@exam...|    4227.73|              0|N#M4NiMHR^|
|11000040|  Dra. Alexia Farias|            da Costa|    AP|ycardoso@example.com|    7843.36|              1|J(A2jexU%7|
|11000058|   Pedro Lucas Jesus|     Campos do Campo|    RN|gcaldeira@example...|   21981.02|              0|P103!Pgjv&|
|11000082|        Rebeca Silva|             Freitas|    RJ|   ilima@example.net|   19198.67|              0|)!Y50WvC1i|
|11000104|   Sra. Maysa Aragão|              Farias|    MA|melissa70@example...|   18552.86|              0|P+_9B(3rrO|
|11000198|        Bruno Novaes|   Barros

##### 2.2.3- Schools

In [43]:
sqlc.sql("""SELECT cod_inep, escola, municipio, uf, telefone, modalidade, categoria12, dependencia, restricao
        FROM tb_school""").show(12)

+--------+--------------------+-----------+---+--------------+--------------------+-------------+-----------+--------------------+
|cod_inep|              escola|  municipio| uf|      telefone|          modalidade|  categoria12|dependencia|           restricao|
+--------+--------------------+-----------+---+--------------+--------------------+-------------+-----------+--------------------+
|11000023|EEEE ABNAEL MACHA...|Porto Velho| RO|(69) 992222374|  Ensino Fundamental|Não Informado|   Estadual|ESCOLA ATENDE EXC...|
|11000040|EMEIEF PEQUENOS T...|Porto Velho| RO| (69) 32274273|   Educação Infantil|Não Informado|  Municipal|ESCOLA EM FUNCION...|
|11000058|CENTRO DE ENSINO ...|Porto Velho| RO| (69) 32244473|Educação Infantil...|   Particular|    Privada|ESCOLA EM FUNCION...|
|11000082|CENTRO EDUCACIONA...|Porto Velho| RO| (69) 32245636|Educação Infantil...|   Particular|    Privada|ESCOLA EM FUNCION...|
|11000104|CENTRO EDUC CORA ...|Porto Velho| RO| (69) 32252616|Educação Infantil...|

24/06/03 02:24:28 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 71 (TID 115): Attempting to kill Python Worker


##### 2.3 - Queries with filters and aggregations

##### 2.3.1 - Query one
 - How many students achieved a literacy fluency level equal to or greater than 60%?

In [18]:
sqlc.sql("""SELECT COUNT(*) as tot_apr FROM tb_student WHERE progresso_alfabetizacao >= 6""").show()

+-------+
|tot_apr|
+-------+
| 444517|
+-------+



##### 2.3.2 - Query two
 - Which teachers are accredited for assessment literacy fluency level?

In [19]:
sqlc.sql("""SELECT esc_cod, pro_nome, pro_cidade, pro_uf, pro_email, pro_credenciado FROM tb_professor WHERE pro_credenciado = 1""").show()

24/06/03 02:13:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/03 02:13:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/03 02:13:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/03 02:13:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/03 02:13:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+--------------------+--------------------+------+--------------------+---------------+
| esc_cod|            pro_nome|          pro_cidade|pro_uf|           pro_email|pro_credenciado|
+--------+--------------------+--------------------+------+--------------------+---------------+
|11000040|  Dra. Alexia Farias|            da Costa|    AP|ycardoso@example.com|              1|
|11000252|     Otávio Caldeira|da Conceição de G...|    TO|cardosoleonardo@e...|              1|
|11000295|      Vinicius Ramos|            da Rocha|    PA|wcardoso@example.org|              1|
|11000309|Pedro Lucas Nogueira|     Freitas da Mata|    AM|joao-guilherme60@...|              1|
|11000350|Luiz Henrique da ...|   Farias das Flores|    GO|bcorreia@example.com|              1|
|11000384|      Stephany Pires|     Campos do Norte|    RO|ryanteixeira@exam...|              1|
|11000422|        Luiza Duarte|      Porto de Pires|    PI|  acunha@example.org|              1|
|11000457|   Emanuella da Rosa

##### 2.3.3 - Query three
 - What are the ages of the students achieved a literacy fluency level equal to or greater than 60%

In [20]:
sqlc.sql("""SELECT idade, count(*) as total FROM tb_student WHERE progresso_alfabetizacao >= 6 
        GROUP BY idade ORDER BY total DESC""").show()

+-----+------+
|idade| total|
+-----+------+
|    6|109023|
|    7|108942|
|    8|108814|
|    5| 62559|
|    9| 55179|
+-----+------+



##### 2.3.4 - Query four
 - How many and how many are the schools per geographic region?

In [21]:
sqlc.sql("""SELECT uf, count(*) as total FROM tb_school GROUP BY uf ORDER BY total DESC""").show(27)

+---+-----+
| uf|total|
+---+-----+
| SP|30580|
| BA|16037|
| MG|15752|
| RJ|11538|
| MA|11436|
| PA|10646|
| RS| 9976|
| PR| 9536|
| PE| 8016|
| CE| 7648|
| SC| 6633|
| AM| 5460|
| GO| 4678|
| PB| 4676|
| PI| 4188|
| RN| 3439|
| ES| 3078|
| AL| 2895|
| MT| 2735|
| SE| 2045|
| MS| 1820|
| TO| 1585|
| AC| 1520|
| DF| 1306|
| RO| 1236|
| RR|  892|
| AP|  879|
+---+-----+



 - What are the schools per geographic region?  (SP as exemple)

In [22]:
sqlc.sql("""SELECT cod_inep, escola, municipio, uf FROM tb_school WHERE uf = 'SP' ORDER BY escola ASC""").show(12)

+--------+--------------------+---------------+---+
|cod_inep|              escola|      municipio| uf|
+--------+--------------------+---------------+---+
|35073581|1 DE MAIO ESCOLA ...|        Guarujá| SP|
|35158562|  10 DE MAIO COLEGIO|      São Paulo| SP|
|35561277|101 DALMATAS CENT...|          Salto| SP|
|35179589|101 DALMATAS ESCO...|Mogi das Cruzes| SP|
|35155226|  13 DE JUNHO ESCOLA| Junqueirópolis| SP|
|35456913|      14 BIS COLEGIO|      São Paulo| SP|
|35169511|       14 BIS ESCOLA|       Campinas| SP|
|35116580| 14 DE AGOSTO ESCOLA|   Mirandópolis| SP|
|35142724| 14 DE JULHO COLEGIO|      São Paulo| SP|
|35234527|15 DE AGOSTO CAIC...|            Jaú| SP|
|35923370|         16 DE JULHO|    Santo André| SP|
|35033881|      17 DE SETEMBRO|        Pompéia| SP|
+--------+--------------------+---------------+---+
only showing top 12 rows



##### 2.3.5 - Query five
 - How many and what types of schools are found in each geographic region?


In [23]:
sqlc.sql("""SELECT uf, dependencia, COUNT(*) AS total_escolas FROM tb_school GROUP BY uf, dependencia ORDER BY uf, dependencia""").show(120)

+---+-----------+-------------+
| uf|dependencia|total_escolas|
+---+-----------+-------------+
| AC|   Estadual|          609|
| AC|    Federal|            7|
| AC|  Municipal|          860|
| AC|    Privada|           44|
| AL|   Estadual|          313|
| AL|    Federal|           17|
| AL|  Municipal|         2040|
| AL|    Privada|          525|
| AM|   Estadual|          757|
| AM|    Federal|           18|
| AM|  Municipal|         4321|
| AM|    Privada|          364|
| AP|   Estadual|          384|
| AP|    Federal|            6|
| AP|  Municipal|          403|
| AP|    Privada|           86|
| BA|   Estadual|         1062|
| BA|    Federal|           37|
| BA|  Municipal|        12071|
| BA|    Privada|         2867|
| CE|   Estadual|          754|
| CE|    Federal|           34|
| CE|  Municipal|         5146|
| CE|    Privada|         1714|
| DF|   Estadual|          700|
| DF|    Federal|           11|
| DF|    Privada|          595|
| ES|   Estadual|          408|
| ES|   

##### 2.3.6 - Query six
    - Which cities and regions can you find second-year students with the best fluency levels?

In [58]:
#rows limited up to cluster resources
dfs_sch_limited = dfs_sch.limit(5000)
dfs_sch_limited = dfs_sch_limited.withColumnRenamed("esc_cod", "esc_cod_inep")  
dfs_std_limited = dfs_std.limit(5000)
dfs_sch_std = dfs_sch_limited.join(dfs_std_limited, dfs_sch_limited.cod_inep == dfs_std_limited.esc_cod, "inner")  
dfs_sch_std.createOrReplaceTempView("tb_school_student")

In [77]:
sqlc.sql("""SELECT uf, municipio, max(progresso_alfabetizacao) AS nota_fluencia FROM tb_school_student 
            WHERE idade >=6 
            GROUP BY uf, municipio
            ORDER BY nota_fluencia DESC, uf ASC""").show(120)

24/06/03 02:37:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/03 02:37:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+--------------------+-------------+
| uf|           municipio|nota_fluencia|
+---+--------------------+-------------+
| AC|     Cruzeiro do Sul|            9|
| AC|         Mâncio Lima|            9|
| AC|Marechal Thaumaturgo|            9|
| AC|        Porto Walter|            9|
| AC|     Rodrigues Alves|            9|
| AC|               Feijó|            9|
| AC|              Jordão|            9|
| AC|            Tarauacá|            9|
| AC|       Manoel Urbano|            9|
| AC| Santa Rosa do Purus|            9|
| AC|      Sena Madureira|            9|
| AC|          Rio Branco|            9|
| AC|              Bujari|            9|
| AC|            Capixaba|            9|
| AC|   Plácido de Castro|            9|
| AC|    Senador Guiomard|            9|
| AC|          Porto Acre|            9|
| AC|        Assis Brasil|            9|
| AC|           Brasiléia|            9|
| AC|      Epitaciolândia|            9|
| AC|              Xapuri|            9|
| AM|          N

24/06/03 02:37:12 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 136 (TID 256): Attempting to kill Python Worker
24/06/03 02:37:12 WARN PythonRunner: Detected deadlock while completing task 1.0 in stage 136 (TID 257): Attempting to kill Python Worker


#### 3 - Generating a SQLite Database

In [54]:
conn = sqlite3.connect('afa-bd.db')

In [55]:
dfs_pro.toPandas().to_sql('tb_professors', con=conn, if_exists='replace')

14000

In [78]:
dfs_std.limit(14000).toPandas().to_sql('tb_students', con=conn, if_exists='replace')

24/06/02 23:28:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/02 23:28:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/02 23:28:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


14000

In [79]:
dfs_sch.toPandas().to_sql('tb_schools', con=conn, if_exists='replace')

180230